# Display velocity vectors on the domain

In [ ]:
import xarray as xr

## Output nc file

In [ ]:
ds = xr.open_mfdataset(f'../tests/data/m1_hello_schism/outputs/out2d_*.nc', concat_dim='time', combine="nested",
                  data_vars='minimal', coords='minimal', compat='override')
ds

In [ ]:
from schimpy import schism_mesh
import numpy as np
import pandas as pd

import hvplot.xarray
import holoviews as hv
from holoviews import opts,dim
hv.extension('bokeh')

import panel as pn
pn.extension()

## Read in hgrid

In [ ]:
smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

dfelems = pd.DataFrame(smesh.elems,columns=[0,1,2])
#dfelems

dfnodes = pd.DataFrame(smesh.nodes, columns=['x','y','z'])
#dfnodes

## Calculate velocity vectors from depth averaged X and Y velocity vectors

In [ ]:
vmag = np.sqrt(ds.depthAverageVelX**2+ds.depthAverageVelY**2)
vangle = np.arctan2(ds.depthAverageVelY,ds.depthAverageVelX)
vel = xr.Dataset({'mag':vmag,'angle': vangle})
vel

## Animate velocity vectors with time

The length of the vectors needs to be scaled depending upon the zoom in and magnitude

In [ ]:
def velocity_field(time, vector_size=1):
    data = vel.isel(time=time)
    vf = hv.VectorField((vel.coords['SCHISM_hgrid_node_x'],vel.coords['SCHISM_hgrid_node_y'],data.angle, data.mag))
    vf.opts(opts.VectorField(pivot='tip', color=dim('Magnitude'), magnitude=dim('Magnitude').norm()*2000*vector_size, rescale_lengths=False))
    return vf.opts(width=600)

dmap = hv.DynamicMap(velocity_field,kdims=['time','vector_size']).opts(title='Velocity Vectors')

hv.output(widget_location='top')

dmap = dmap.redim.range(time=(0,len(vel.time)-1)).redim.values(vector_size=[0.25,0.5,0.75,1,2,5,10,20])

In [ ]:
dash = pn.Row(dmap)

In [ ]:
dash.servable('Velocity Vector Animations')#.show()